In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Read data 
SP500 = pd.read_csv('sp500/SP500.csv', parse_dates=['Date'], index_col='Date')
stocks = pd.read_csv('sp500_joined_closes.csv', parse_dates=['Date'], index_col='Date')

In [3]:
# Drop columns other then Adj Close
SP500.drop(['Open','High','Low','Close','Volume'], 1, inplace=True)
# Rename Adj Close column to 'SP500'
SP500.rename(columns = {'Adj Close': 'SP500'}, inplace=True)

SP500.head()

,SP500
Date,
2018-01-02,2695.810059
2018-01-03,2713.060059
2018-01-04,2723.989990
2018-01-05,2743.149902
2018-01-08,2747.709961


In [4]:
# Turn Adj Closes into returns
stock_returns = stocks.pct_change().mul(100)
SP500_returns = SP500.pct_change().mul(100)

In [5]:
SP500_returns.head()

,SP500
Date,
2018-01-02,NaN
2018-01-03,0.639882
2018-01-04,0.402864
2018-01-05,0.703377
2018-01-08,0.166234


In [6]:
# Slice the data from 2019-2020
stock_returns = stock_returns['2019':'2020']
SP500_returns = SP500_returns['2019':'2020']
stocks = stocks['2019':'2020']

In [7]:
SP500_returns.head()

,SP500
Date,
2019-01-02,0.126850
2019-01-03,-2.475673
2019-01-04,3.433571
2019-01-07,0.701043
2019-01-08,0.969529


In [8]:
# Choose 25 securities to build our portfolios from
tickers = ['WBA','ZION','IRM','JPM','ADP','HAL','KMX','VZ','MCD','ALLE','PLD',
          'PG','AEP','MMM','ACN','HBAN','HCA','BBY','KMB','CMI','YUM','RF',
          'HON','DD','NOW']

stock_returns = stock_returns[tickers]
stocks = stocks[tickers]
stock_returns.head()

,WBA,ZION,IRM,JPM,ADP,HAL,KMX,VZ,MCD,ALLE,...,HBAN,HCA,BBY,KMB,CMI,YUM,RF,HON,DD,NOW
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,-0.420626,2.076843,-0.939457,1.735546,-0.748385,2.687747,0.382592,-0.363776,-0.852089,1.670705,...,1.805054,-2.163089,1.203631,-1.888015,-0.197161,-0.516274,2.095085,-0.220247,1.943487,0.151643
2019-01-03,-1.048185,-0.559512,1.580611,-1.418062,-3.016112,-0.615858,-1.715102,0.365104,-0.657895,-1.680883,...,-0.797872,-1.902219,-2.047183,-1.028205,-1.975504,-2.515794,-0.078927,-1.340166,-3.691184,-5.159264
2019-01-04,3.335968,3.069054,1.763485,3.688081,3.985596,4.763749,2.779124,0.248899,1.933059,-0.229650,...,2.770331,3.308386,0.477707,1.115135,4.151552,2.603865,3.633491,3.499800,4.225748,5.978004
2019-01-07,0.566095,0.397022,1.087326,0.062585,-0.543051,1.404806,4.433265,0.630252,1.088674,1.048593,...,0.434783,0.847946,5.705230,1.357338,0.495356,-0.112790,1.676829,0.555856,0.053879,4.151091
2019-01-08,2.008215,0.914483,2.218487,-0.187637,0.593495,1.312432,-2.288123,2.941735,0.208442,1.379398,...,0.952381,1.404082,2.942279,1.367060,0.261861,-0.203252,1.049475,-0.207294,1.157781,0.771415


In [9]:
# Function to calculate returns & volatility of portfolios
def portfolio_performance(weights, mean_returns, cov_matrix):
    #annualized returns
    returns = np.sum(mean_returns*weights)*252
    #annualized std
    std = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights)))*np.sqrt(252)
    
    return std, returns

In [10]:
# generate random portfolios with random weights for each security
def generate_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    #create results array, 3 by N, (3 for std dev, return, and sharpe ratio)
    results = np.zeros((3,num_portfolios))
    weight_array = []
    
    for i in range(num_portfolios):
        # Set seed for replicability
        np.random.seed(100+i)
        weights = np.random.random(25)
        weights /= np.sum(weights)
        weight_array.append(weights)
        
        portfolio_std_dev, portfolio_return = portfolio_performance(weights, 
                                                                    mean_returns, 
                                                                    cov_matrix) 
        results[0,i]= portfolio_std_dev
        results[1,i]= portfolio_return
        #sharpe ratio
        results[2,i]= (portfolio_return - risk_free_rate)/portfolio_std_dev
        
    return results, weight_array
        

In [11]:
# Set up the input variables, recall our returns are stock_returns
mean_returns = stock_returns.mean()
cov_matrix = stock_returns.cov()
num_portfolios = 100000
# Remark: one way to improve this would be download historical data for risk free rate,
# such as treasury bill. Since over a year the rate is relatively constant this 
# simplication is fine
risk_free_rate = 0.02

In [12]:
def display_portfolios(mean_returns, cov_matrix, num_portfolios, risk_free_rate,
                       SP500_returns):
    
    results, weights = generate_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate)
    
    ### Identify portfolio with max sharpe ratio (MSR Portfolio)
    max_sharpe_portfolio = np.argmax(results[2])
    # Get the corresponding std dev and returns
    max_sharpe_std = results[0, max_sharpe_portfolio]
    max_sharpe_returns = results[1, max_sharpe_portfolio]
    max_sharpe_sharpe = results[2, max_sharpe_portfolio]
    # Get the weights associated with the MSR, round numbers, transpose for later use
    max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_portfolio], index=stocks.columns,
                                        columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2) for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    
    ### Identify portfolio with minimum volatility (GMV Portfolio)
    gmv_portfolio = np.argmin(results[0])
    # Corresponding std dev and returns
    gmv_std = results[0, gmv_portfolio]
    gmv_returns = results[1, gmv_portfolio]
    gmv_sharpe = results[2, gmv_portfolio]
    # Weights
    gmv_allocation = pd.DataFrame(weights[gmv_portfolio], index=stocks.columns, columns=['allocation'])
    gmv_allocation.allocation = [round(i*100,2) for i in gmv_allocation.allocation]
    gmv_allocation = gmv_allocation.T
    
    # Uniform weight portfolio
    num_stocks = 25
    equal_weights = np.repeat(1/num_stocks, num_stocks)
    equal_portfolio = mean_returns*equal_weights
    equal_port_returns = np.sum(mean_returns*equal_weights)*252
    equal_port_std = np.sqrt(np.dot(equal_weights.T,np.dot(cov_matrix, equal_weights)))*np.sqrt(252)
    equal_port_sharpe = (equal_port_returns-risk_free_rate)/equal_port_std
    
    equal_allocation = pd.DataFrame(equal_weights, index=stocks.columns, columns=['allocation'])
    equal_allocation.allocation = [round(i*100,2) for i in equal_allocation.allocation]
    equal_allocation = equal_allocation.T
    
    # S&P500
    SP500_weight = np.repeat(1,1)
    SP500_portfolio = SP500_returns*SP500_weight
    SP500_returns = np.sum(SP500_portfolio)
    SP500_std = np.std(SP500_portfolio)*np.sqrt(252)
    SP500_sharpe = (SP500_returns-risk_free_rate)/SP500_std
    
    
    # Print results
    # this first setting ensures we do not truncate our frame when printing
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    print("*"*50)
    print("\n")
    print("*** Max Sharpe Ratio Portfolio (MSR) ***\n")
    print("* Returns, Volatility *\n")
    print("Annualized Return:", round(max_sharpe_returns, 2))
    print("Annualized Volatility:", round(max_sharpe_std, 2))
    print("Sharpe Ratio:", round(max_sharpe_sharpe, 2))
    print("\n")
    print("* Allocation of MSR Portfolio *\n")
    print(max_sharpe_allocation)
    print("\n")
    print("*"*50)

    print("\n")
    print("*** Global Minimum Volatility Portfolio (GMV) ***\n")
    print("* Returns, Volatility *\n")
    print("Annualized Return:", round(gmv_returns, 2))
    print("Annualized Volatility:", round(gmv_std, 2))
    print("Sharpe Ratio:", round(gmv_sharpe, 2))
    print("\n")
    print("* Allocation of GMV Portfolio *\n")
    print(gmv_allocation)
    print("\n")
    print("*"*50)
    
    print("\n")
    print("*** Equal Weighted Portfolio ***\n")
    print("* Returns, Volatility *\n")
    print("Annualized Return:", round(equal_port_returns, 2))
    print("Annualized Volatility:", round(equal_port_std, 2))
    print("Sharpe Ratio:", round(equal_port_sharpe, 2))
    print("\n")
    print("* Allocation of Equal Weighted Portfolio *\n")
    print(equal_allocation)
    print("\n")
    print("*"*50)
    
    print("\n")
    print("*** S&P 500 ***\n")
    print("* Returns, Volatility *\n")
    print("Annualized Return:", round(SP500_returns[0], 2))
    print("Annualized Volatility:", round(SP500_std[0], 2))
    print("Sharpe Ratio:", round(SP500_sharpe[0], 2))
    print("\n")
    print("*"*50)
    

    # Scatter Plot
    %matplotlib qt
    plt.style.use('seaborn-colorblind')  
    plt.figure(figsize=(16,9))
    # plt.scatter(x is volatility, y is returns, color by sharpe ratio)
    plt.scatter(results[0,:],results[1,:],c=results[2,:], cmap='YlGn', marker='o', s=10, alpha=0.8)
    plt.colorbar()
    
    # Mark MSR on scatter
    plt.scatter(max_sharpe_std, max_sharpe_returns, marker='x',color='red', s=125, label= 'MSR')
    
    # Mark GMV on scatter
    plt.scatter(gmv_std, gmv_returns, marker='x',color='blue', s=125, label= 'GMV')
    
    # Mark Equal on scatter
    plt.scatter(equal_port_std, equal_port_returns, marker='x', color='black', s=125, label='Equal Weights')
    
    # Mark SP500 on scatter
    plt.scatter(SP500_std, SP500_returns, marker='x', color='darkorange', s=125, label='S&P 500')
    
    plt.title('Portfolio Simulation')
    plt.xlabel('Annualized Volatility')
    plt.ylabel('Annualized Returns')
    plt.legend(labelspacing=1.2)
    
    
display_portfolios(mean_returns, cov_matrix, num_portfolios, risk_free_rate, 
                   SP500_returns)

**************************************************


*** Max Sharpe Ratio Portfolio (MSR) ***

* Returns, Volatility *

Annualized Return: 33.81
Annualized Volatility: 11.63
Sharpe Ratio: 2.91


* Allocation of MSR Portfolio *

            WBA  ZION   IRM   JPM   ADP   HAL   KMX    VZ   MCD  ALLE   PLD  \
allocation  1.4  7.42  0.81  5.61  1.26  0.55  4.43  6.31  5.35   7.3  7.47   

              PG   AEP   MMM  ACN  HBAN   HCA   BBY  KMB   CMI   YUM    RF  \
allocation  4.82  6.65  0.58  6.6  1.95  1.95  7.43  7.2  1.08  0.76  5.34   

             HON   DD   NOW  
allocation  1.06  1.2  5.44  


**************************************************


*** Global Minimum Volatility Portfolio (GMV) ***

* Returns, Volatility *

Annualized Return: 26.1
Annualized Volatility: 10.26
Sharpe Ratio: 2.54


* Allocation of GMV Portfolio *

             WBA  ZION  IRM   JPM   ADP   HAL   KMX    VZ   MCD  ALLE   PLD  \
allocation  3.94  1.33  9.1  1.25  4.45  0.59  4.27  8.03  4.09  3.57  2.38   
